In [1]:
from openfisca_us import Microsimulation, IndividualSim

In [2]:
from openfisca_us.model_api import *
from openfisca_us.tools.baseline_variables import baseline_variables

REPLACED_PROGRAMS = ["ctc", "eitc"]

CHILD_AMOUNT = 393
CHILD_AGE_LIMIT = 18
ADULT_DEPENDENT_CREDIT = 600
FILER_CREDIT_SINGLE = 600
FILER_CREDIT_MARRIED = 1200
FILER_CREDIT_SINGLE_PHASE_OUT_START = 20_000
FILER_CREDIT_MARRIED_PHASE_OUT_START = 40_000
FILER_CREDIT_PHASE_OUT_RATE = 0.05
FILER_CREDIT_MIN_AGE = 19
FILER_CREDIT_MAX_AGE = 64

class ecpa(Variable):
    value_type = float
    entity = TaxUnit
    definition_period = YEAR
    label = "End Child Poverty Act"

    formula = sum_of_variables(['ecpa_child_allowance', 'ecpa_adult_dependent_credit', 'ecpa_filer_credit'])

class ecpa_child_allowance(Variable):
    value_type = float
    entity = TaxUnit
    definition_period = YEAR
    label = "End Child Poverty Act Child Allowance"

    def formula(tax_unit, period, parameters):
        person = tax_unit.members
        # Child allowance.
        qualifies_for_child_allowance = person("age", period) <= CHILD_AGE_LIMIT
        child_allowance = CHILD_AMOUNT * tax_unit.sum(qualifies_for_child_allowance) * 12
        return child_allowance

class ecpa_adult_dependent_credit(Variable):
    value_type = float
    entity = TaxUnit
    definition_period = YEAR
    label = "End Child Poverty Act Adult Dependent Credit"

    def formula(tax_unit, period, parameters):
        person = tax_unit.members
        # Adult dependent credit.
        adult_dependent = person("is_tax_unit_dependent", period) & ~(person("age", period) <= CHILD_AGE_LIMIT)
        adult_dependent_credit = ADULT_DEPENDENT_CREDIT * tax_unit.sum(adult_dependent)
        return adult_dependent_credit

class ecpa_filer_credit(Variable):
    value_type = float
    entity = TaxUnit
    definition_period = YEAR
    label = "End Child Poverty Act Filer Credit"

    def formula(tax_unit, period, parameters):
        # Filer credit.
        # Define eligibility based on age.
        age_head = tax_unit("age_head", period)
        age_spouse = tax_unit("age_spouse", period)
        head_qualifies = (age_head >= FILER_CREDIT_MIN_AGE) & (age_head <= FILER_CREDIT_MAX_AGE)
        spouse_qualifies = (age_spouse >= FILER_CREDIT_MIN_AGE) & (age_spouse <= FILER_CREDIT_MAX_AGE)
        filer_credit_eligible = head_qualifies | spouse_qualifies
        # Get maximum amount.
        filing_status = tax_unit("filing_status", period)
        is_married = filing_status == filing_status.possible_values.JOINT
        max_filer_credit = where(is_married, FILER_CREDIT_MARRIED, FILER_CREDIT_SINGLE)
        # Phase out.
        agi = tax_unit("adjusted_gross_income", period)
        phase_out_start = where(is_married, FILER_CREDIT_MARRIED_PHASE_OUT_START, FILER_CREDIT_SINGLE_PHASE_OUT_START)
        excess = max_(agi - phase_out_start, 0)
        reduction = excess * FILER_CREDIT_PHASE_OUT_RATE
        # Compute final amount.
        filer_credit = filer_credit_eligible * max_(max_filer_credit - reduction, 0)
        return filer_credit

class spm_unit_net_income(Variable):
    def formula(spm_unit, period, parameters):
        original = baseline_variables["spm_unit_net_income"].formula(spm_unit, period, parameters)
        return original + add(spm_unit, period, ["ecpa"])

class add_ecpa(Reform):
    def apply(self):
        for program in REPLACED_PROGRAMS:
            self.neutralize_variable(program)
        self.add_variable(ecpa_child_allowance)
        self.add_variable(ecpa_adult_dependent_credit)
        self.add_variable(ecpa_filer_credit)
        self.add_variable(ecpa)
        self.update_variable(spm_unit_net_income)

In [16]:
isb = IndividualSim(year=2022)
isr = IndividualSim(add_ecpa, year=2022)

for sim in [isb, isr]:
    members = ["head", "child_1", "child_2", "dependent"]
    sim.add_person(name="head", age=27, employment_income=30000, is_tax_unit_head=True, is_tax_unit_dependent=False)
    sim.add_person(name="child_1", age=10, is_tax_unit_dependent=False)
    sim.add_person(name="child_2", age=10, is_tax_unit_dependent=False)
    sim.add_person(name="dependent", age=56, is_tax_unit_spouse=False, is_tax_unit_dependent=True)
    sim.add_tax_unit(name="tax_unit", members=members, premium_tax_credit=0)
    sim.add_spm_unit(name="spm_unit", members=members, snap_emergency_allotment=0)
    sim.add_household(name="household", members=members, state_code="MA")


In [18]:
isr.calc('ecpa_child_allowance')

array([9432.], dtype=float32)